# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [23]:
# Dependencies and Setup

import pandas as pd
import requests


In [24]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [25]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [26]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [27]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [28]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_13396\3249227210.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1332,1337,1344
unique,53,24,301,22,61,561,383,411,538,911,66,278,6,30,86,1126,2
top,US-CAL-BANC,2024-07-17T02:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,0,70,True
freq,48,56,819,1306,1200,619,758,666,348,192,1109,539,1325,1293,207,4,1069


In [29]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 
           'battery_discharge':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,1148.269345,13.603423,19.620536,1774.866815,586.507440,497.973958,652.872024,4016.080357,9.327381,87.896577,0.248512,21.203869
std,2484.387934,101.125025,98.230042,4412.157710,2086.241177,1961.350647,1461.792182,8214.610454,53.001088,177.505031,4.473717,342.057330
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,260.750000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,47.000000,0.000000,0.000000,59.500000,980.500000,0.000000,10.000000,0.000000,0.000000
75%,405.750000,0.000000,0.000000,1521.750000,158.000000,192.250000,692.000000,2556.000000,0.000000,89.250000,0.000000,0.000000
max,13324.000000,811.000000,596.000000,32755.000000,18360.000000,17186.000000,11538.000000,51881.000000,864.000000,1416.000000,103.000000,7075.000000


In [30]:
# convert measured Energy values to integers in Giga Watts
convert_dict = {'hydro': int}
#convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
             #   'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               # }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int64
geothermal               int64
biomass                  int64
coal                     int64
wind                     int64
solar                    int64
hydro                    int32
gas                      int64
oil                      int64
unknown                  int64
hydro-discharge          int64
battery_discharge        int64
renewable_percentage    object
total_consumption       object
estimated               object
dtype: object

In [31]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [32]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-07-17 02:00:00,US-SW-PNM,0,0,0,188,1090,1,17,629,0,206,0,0,52,2131,False,02:00:00,2024-07-17
2024-07-17 03:00:00,US-SW-PNM,0,0,0,187,1102,0,17,575,0,176,0,0,54,2057,False,03:00:00,2024-07-17
2024-07-17 04:00:00,US-SW-PNM,0,0,0,187,889,0,17,572,0,96,0,0,51,1761,False,04:00:00,2024-07-17
2024-07-17 05:00:00,US-SW-PNM,0,0,0,184,1025,0,17,575,0,6,0,0,58,1807,False,05:00:00,2024-07-17
2024-07-17 06:00:00,US-SW-PNM,0,0,0,95,1276,0,17,575,0,0,0,0,66,1963,False,06:00:00,2024-07-17


Section 3
Transform carbon intensity data

In [33]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [34]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_13396\3332989139.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [35]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [36]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [37]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-07-17 02:00:00,US-SW-PNM,0,0,0,188,1090,1,17,629,0,206,0,0,52,2131,False,02:00:00,2024-07-17,342,False
2024-07-17 03:00:00,US-SW-PNM,0,0,0,187,1102,0,17,575,0,176,0,0,54,2057,False,03:00:00,2024-07-17,329,False
2024-07-17 04:00:00,US-SW-PNM,0,0,0,187,889,0,17,572,0,96,0,0,51,1761,False,04:00:00,2024-07-17,351,False
2024-07-17 05:00:00,US-SW-PNM,0,0,0,184,1025,0,17,575,0,6,0,0,58,1807,False,05:00:00,2024-07-17,307,False
2024-07-17 06:00:00,US-SW-PNM,0,0,0,95,1276,0,17,575,0,0,0,0,66,1963,False,06:00:00,2024-07-17,225,False


In [38]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4466.000000,4483.000000,4515.000000
mean,1223.636766,16.659136,27.557697,1760.921595,825.968771,536.102990,687.005094,4287.144408,13.620598,96.389147,0.149945,22.182503,33.130766,9565.234664,289.694131
std,2519.165008,111.545525,119.238846,4561.155559,2817.863105,2117.982879,1429.473392,8331.244638,84.571380,204.240515,3.200697,306.309625,34.263734,17771.598432,89.939566
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,227.000000,0.000000,0.000000,0.000000,0.000000,5.000000,901.500000,220.000000
50%,0.000000,0.000000,0.000000,37.000000,0.000000,0.000000,73.000000,973.000000,0.000000,10.000000,0.000000,0.000000,21.000000,2377.000000,293.000000
75%,1026.000000,0.000000,0.000000,1394.000000,235.500000,194.000000,769.500000,3293.000000,0.000000,94.500000,0.000000,0.000000,51.000000,9531.000000,340.000000
max,13614.000000,812.000000,657.000000,34899.000000,21642.000000,19768.000000,11331.000000,54713.000000,2054.000000,1864.000000,103.000000,7075.000000,100.000000,117668.000000,598.000000


In [39]:
df_us_energy_reindex.drop_duplicates(inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4515.000000,4466.000000,4483.000000,4515.000000
mean,1223.636766,16.659136,27.557697,1760.921595,825.968771,536.102990,687.005094,4287.144408,13.620598,96.389147,0.149945,22.182503,33.130766,9565.234664,289.694131
std,2519.165008,111.545525,119.238846,4561.155559,2817.863105,2117.982879,1429.473392,8331.244638,84.571380,204.240515,3.200697,306.309625,34.263734,17771.598432,89.939566
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,227.000000,0.000000,0.000000,0.000000,0.000000,5.000000,901.500000,220.000000
50%,0.000000,0.000000,0.000000,37.000000,0.000000,0.000000,73.000000,973.000000,0.000000,10.000000,0.000000,0.000000,21.000000,2377.000000,293.000000
75%,1026.000000,0.000000,0.000000,1394.000000,235.500000,194.000000,769.500000,3293.000000,0.000000,94.500000,0.000000,0.000000,51.000000,9531.000000,340.000000
max,13614.000000,812.000000,657.000000,34899.000000,21642.000000,19768.000000,11331.000000,54713.000000,2054.000000,1864.000000,103.000000,7075.000000,100.000000,117668.000000,598.000000


In [40]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,5071.000000,5071.000000,5071.000000,5071.00000,5071.000000,5071.000000,5071.000000,5071.000000,5071.000000,5071.000000,5071.000000,5071.000000,5071.000000
mean,1232.644843,16.394005,27.140801,1800.50069,791.799842,585.813252,700.374680,4364.818379,13.554329,97.221455,0.149872,20.169197,276.184382
std,2527.982894,110.379613,118.023392,4641.69007,2717.753070,2210.327901,1466.632194,8496.336809,83.183695,203.591242,3.236871,289.901027,94.291636
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,232.000000,0.000000,0.000000,0.000000,0.000000,210.000000
50%,0.000000,0.000000,0.000000,38.00000,0.000000,1.000000,76.000000,986.000000,0.000000,10.000000,0.000000,0.000000,257.000000
75%,1051.000000,0.000000,0.000000,1401.50000,226.000000,239.500000,782.500000,3537.500000,0.000000,95.000000,0.000000,0.000000,337.000000
max,13614.000000,812.000000,657.000000,34899.00000,21642.000000,19768.000000,11538.000000,54713.000000,2054.000000,1864.000000,103.000000,7075.000000,598.000000


In [41]:
#df_both.to_csv(f'data/{dates[0]}USenergy_data2.csv')
df_both.to_csv(f'data/runningUSenergy_data.csv')

In [42]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.00000,1810.000000
mean,1271.092818,44.248619,71.592818,1937.750276,1235.141989,768.818232,1097.120442,5495.023204,29.646961,117.035359,0.001105,53.68895,313.524309
std,2197.892541,177.933134,183.251335,4922.797207,3315.628718,2831.712899,1724.944004,8814.527087,125.874329,230.113909,0.033232,476.13593,91.709227
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,215.000000,0.000000,0.000000,0.000000,0.00000,231.250000
50%,0.000000,0.000000,0.000000,30.000000,30.000000,1.000000,217.000000,1161.500000,0.000000,15.000000,0.000000,0.00000,314.000000
75%,2198.000000,0.000000,2.750000,1195.500000,463.000000,217.250000,1345.000000,8934.750000,0.000000,128.000000,0.000000,0.00000,352.000000
max,13263.000000,812.000000,657.000000,34149.000000,21642.000000,19768.000000,10136.000000,54713.000000,2054.000000,1864.000000,1.000000,7075.00000,598.000000


In [43]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filtered.csv')